In [63]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.s3 import S3Uploader
from sagemaker.pytorch import PyTorch

import os
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 

In [64]:
sagemaker_session = sagemaker.Session()
role = get_execution_role()
region = sagemaker_session.boto_session.region_name

In [67]:
train_uri = S3Uploader.upload("../data/churn.csv", 's3://responsibleai/data')
output_path = 's3://responsibleai/dp/output'
checkpoint_local_path="/opt/ml/checkpoints"
checkpoint_s3_bucket = 's3://responsibleai/dp/checkpoint'

In [68]:
pytorch_estimator = PyTorch(entry_point = '../code/train.py',
                            source_dir = '../code',
                            instance_type='ml.p3.2xlarge',
                            output_path = 's3://responsibleai/dp/output',
                            checkpoint_s3_uri=checkpoint_s3_bucket,
                            checkpoint_local_path=checkpoint_local_path,
                            role = role,
                            instance_count=1,
                            framework_version='1.8.0',
                            py_version='py3',
                            hyperparameters = {'epochs': 100, 
                                               'batch-size': 64, 
                                               'learning-rate': 0.003})

In [69]:
pytorch_estimator.fit({'train': 's3://responsibleai/data/churn.csv'})

2022-03-02 22:49:34 Starting - Starting the training job...ProfilerReport-1646261374: InProgress
...
2022-03-02 22:50:31 Starting - Preparing the instances for training......
2022-03-02 22:51:33 Downloading - Downloading input data...
2022-03-02 22:51:58 Training - Downloading the training image...........................
2022-03-02 22:56:26 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-03-02 22:56:29,763 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-03-02 22:56:29,789 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-03-02 22:56:29,797 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-03-02 22:56:30,779 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.6 -m pip install -

In [24]:
model_name = "churn-clarify-model" 

predictor = pytorch_estimator.deploy(initial_instance_count=1, 
                         instance_type='ml.p3.2xlarge',
                         model_name = "churn-clarify-model")

Using already existing model: churn-clarify-model


---------!

In [47]:
predictor.delete_endpoint()